In this notebook, we're going to implement Convo2D from scratch

In [5]:
import numpy as np

In [6]:
def conv2d(input, kernel, padding = 0, stride = 1):
    input_height, input_width = input.shape
    kernel_height, kernel_width = kernel.shape

    #calculate output dimentsions
    output_height = (input_height - kernel_height + 2*padding)//stride +1
    output_width = (input_width - kernel_width + 2*padding)//stride +1

    #padding to input if needed
    if padding > 0:
        padded_input = np.zeros((input_height + 2*padding, input_width + 2*padding))
        padded_input[padding:padding + input_height, padding: padding + input_width] = input
    else:
        padded_input = input

    #initilize output
    output = np.zeros((output_height, output_width))

    #perform convulation
    for i in range(output_height):
        for j in range (output_width):
            output[i,j] = np.sum(padded_input[i*stride: i*stride + kernel_height, j*stride: j*stride + kernel_width] * kernel)

    return output

input_matrix = np.array([
    [1,2,3,0,0],
    [4,5,6,0,0],
    [7,8,9,0,0],
    [0,0,0,0,0],
    [0,0,0,0,0]
])

kernel_matrix = np.array([
    [1,0,1],
    [1,0,1],
    [1,0,1]
])

output_matrix = conv2d(input_matrix, kernel_matrix, padding = 1, stride = 2)
print("Our Conv2D: \n")
print(output_matrix)

Our Conv2D: 

[[ 7.  7.  0.]
 [13. 13.  0.]
 [ 0.  0.  0.]]


Now, we code this using PyTorch

In [7]:
import torch
import torch.nn as nn

input_tensor = torch.tensor(input_matrix, dtype = torch.float32).unsqueeze(0).unsqueeze(0)
kernel_tensor = torch.tensor(kernel_matrix, dtype = torch.float32).unsqueeze(0).unsqueeze(0)

conv = nn.Conv2d(in_channels = 1, out_channels = 1, kernel_size = 3, padding = 1, stride = 2, bias = False)
conv.weight.data = kernel_tensor

output_tensor = conv(input_tensor)
output_pytorch = output_tensor.squeeze().detach().numpy()

print("PyTorch conv2d output: \n")
print(output_pytorch)


PyTorch conv2d output: 

[[ 7.  7.  0.]
 [13. 13.  0.]
 [ 0.  0.  0.]]


Putting it all together with PyTorch

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from IPython.display import Javascript
from sklearn.metrics import ConfusionMatrixDisplay

In [21]:
import torch
import torchvision 
import torchvision.transforms as transforms
from torch.utils import data 
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim

In [17]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5,0.5,0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train = True, download = False, transform = transform)
trainloader = DataLoader(trainset, batch_size=batch_size,
                         shuffle=True, num_workers=2) 

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car','bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
print("Datast and DataLoaders configured successsfully")

Datast and DataLoaders configured successsfully


In [19]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
print("Network initialized successfully")


Network initialized successfully


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [22]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

[1,  2000] loss: 2.239
[1,  4000] loss: 1.850
[1,  6000] loss: 1.689
[1,  8000] loss: 1.551
[1, 10000] loss: 1.518
[1, 12000] loss: 1.450
[2,  2000] loss: 1.399
[2,  4000] loss: 1.375
[2,  6000] loss: 1.347
[2,  8000] loss: 1.335
[2, 10000] loss: 1.293
[2, 12000] loss: 1.270
